In [3]:
import warnings
warnings.filterwarnings("ignore")

from scipy import constants as cst
from pint import UnitRegistry
ureg = UnitRegistry()
import sys
import multiprocessing
import subprocess
import numpy as np
import shutil
import os

path_to_code = "generated-codes/chapter6/"
sys.path.append(path_to_code)

from MinimizeEnergy import MinimizeEnergy
from MonteCarlo import MonteCarlo
from WriteLAMMPSfiles import write_topology_file, write_lammps_parameters, write_lammps_variables

"The values of the potential constants for argon have been taken throughout as  
E*/k= 119.76°K, r*=3.822 A, v*=23.79 cm3/mole, as determined by Michels6 from  
second virial coefficient data." [1]  

[1] Wood and Parker. The Journal of Chemical Physics, 27(3):720–733, 1957.  

In [4]:
kB = cst.Boltzmann*ureg.J/ureg.kelvin # boltzman constant
Na = cst.Avogadro/ureg.mole # avogadro
R = kB*Na # gas constant

In [7]:
def launch_MC_code(tau):

    epsilon = (119.76*ureg.kelvin*kB*Na).to(ureg.kcal/ureg.mol) # kcal/mol
    r_star = 3.822*ureg.angstrom # angstrom
    sigma = r_star / 2**(1/6) # angstrom
    N_atom = 200 # no units
    m_argon = 39.948*ureg.gram/ureg.mol
    T = (273.15+55)*ureg.kelvin # 55°C
    volume_star = (23.79 * ureg.centimeter**3/ureg.mole).to(ureg.angstrom**3/ureg.mole)
    cut_off = sigma*2.5
    displace_mc = sigma/5 # angstrom
    volume = N_atom*volume_star*tau/Na
    box_size = volume**(1/3)
    folder = "outputs_tau"+str(tau)+"/"

    em = MinimizeEnergy(maximum_steps=100,
                        thermo_period=10,
                        dumping_period=10,
        number_atoms=[N_atom],
        epsilon=[epsilon.magnitude], 
        sigma=[sigma.magnitude],
        atom_mass=[m_argon.magnitude],
        box_dimensions=[box_size.magnitude,
                        box_size.magnitude,
                        box_size.magnitude],
        cut_off=cut_off.magnitude,
        data_folder=folder,
    )
    em.run()

    mc = MonteCarlo(maximum_steps=2000,
        dumping_period=100,
        thermo_period=100,
        neighbor=50,
        displace_mc = displace_mc.magnitude,
        desired_temperature = T.magnitude,
        number_atoms=[N_atom],
        epsilon=[epsilon.magnitude], 
        sigma=[sigma.magnitude],
        atom_mass=[m_argon.magnitude],
        box_dimensions=[box_size.magnitude,
                        box_size.magnitude,
                        box_size.magnitude],
        initial_positions = em.atoms_positions*em.reference_distance,
        cut_off=cut_off.magnitude,
        data_folder=folder,
    )
    mc.run()

    folder = "lammps_tau"+str(tau)+"/"
    if os.path.exists(folder) is False:
        os.mkdir(folder)

    write_topology_file(mc, filename=folder+"initial.data")
    write_lammps_parameters(mc, filename=folder+"PARM.lammps")
    write_lammps_variables(mc, filename=folder+"variable.lammps")

    mycwd = os.getcwd() # initial path
    os.chdir(folder)
    shutil.copyfile("../lammps/input.lmp", "input.lmp")
    subprocess.call(["/home/simon/Softwares/lammps-2Aug2023/src/lmp_serial", "-in", "input.lmp"])
    os.chdir(mycwd)

In [8]:
if __name__ == "__main__":
    tau_values = np.round(np.logspace(-0.126, 0.882, 10),2)
    pool = multiprocessing.Pool()
    squared_numbers = pool.map(launch_MC_code, tau_values)
    pool.close()
    pool.join()

step epot maxFstep epot maxFstep epot maxF


0 183973535.966 3024391804.9980 24190441.100 320167252.1280 8792870.743 104743817.691


step epot maxF
0 3205350.726 34438888.340
step epot maxF
0 1360333564.848 27916995354.277
step epot maxFstep epot maxF
step epot maxF
0 506320561.545 9302489816.8200 419873.002 3684772.747


0 1159785.745 11250831.062step epot maxF

0 3699323424.281 85103921180.426
step epot maxF
0 66663847.809 982296297.274
10 201713.039 336622.475
10 889772.631 1429693.741
10 96128.255 159345.971
10 47693.521 79757.877
10 22988.964 39016.553
10 8128192.918 11774158.145
10 10744.153 18634.086
10 3976024.881 6014862.286
10 429197.846 715077.370
10 1931170.761 3054226.361
20 -57.586 1.575
20 -22.200 29.364
20 -70.431 3.214
20 17255.753 6258.783
20 969.935 447.750
20 -62.419 14.926
20 -46.638 0.951
20 6997.814 1253.493
30 -70.668 0.574
20 2185.177 422.074
30 -129.139 7.799
20 243.520 248.427
30 -194.538 5.385
30 -84.909 0.608
30 -105.325 1.385
40 -145.539 2.006
30 -89.846 5